Lambda School Data Science

*Unit 2, Sprint 1, Module 2*

---

# Regression 2

## Assignment

You'll continue to **predict how much it costs to rent an apartment in NYC,** using the dataset from renthop.com.

- [x] Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.
- [x] Engineer at least two new features. (See below for explanation & ideas.)
- [x] Fit a linear regression model with at least two features.
- [x] Get the model's coefficients and intercept.
- [x] Get regression metrics RMSE, MAE, and $R^2$, for both the train and test data.
- [ ] What's the best test MAE you can get? Share your score and features used with your cohort on Slack!
- [ ] As always, commit your notebook to your fork of the GitHub repo.


#### [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)

> "Some machine learning projects succeed and some fail. What makes the difference? Easily the most important factor is the features used." — Pedro Domingos, ["A Few Useful Things to Know about Machine Learning"](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)

> "Coming up with features is difficult, time-consuming, requires expert knowledge. 'Applied machine learning' is basically feature engineering." — Andrew Ng, [Machine Learning and AI via Brain simulations](https://forum.stanford.edu/events/2011/2011slides/plenary/2011plenaryNg.pdf) 

> Feature engineering is the process of using domain knowledge of the data to create features that make machine learning algorithms work. 

#### Feature Ideas
- Does the apartment have a description?
- How long is the description?
- How many total perks does each apartment have?
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths)
- Ratio of beds to baths
- What's the neighborhood, based on address or latitude & longitude?

## Stretch Goals
- [ ] If you want more math, skim [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf),  Chapter 3.1, Simple Linear Regression, & Chapter 3.2, Multiple Linear Regression
- [ ] If you want more introduction, watch [Brandon Foltz, Statistics 101: Simple Linear Regression](https://www.youtube.com/watch?v=ZkjP5RJLQF4)
(20 minutes, over 1 million views)
- [ ] Add your own stretch goal(s) !

In [103]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [104]:
import numpy as np
import pandas as pd

# Read New York City apartment rental listing data
df = pd.read_csv(DATA_PATH+'apartments/renthop-nyc.csv')
assert df.shape == (49352, 34)

# Remove the most extreme 1% prices,
# the most extreme .1% latitudes, &
# the most extreme .1% longitudes
df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
        (df['price'] <= np.percentile(df['price'], 99.5)) & 
        (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
        (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
        (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
        (df['longitude'] <= np.percentile(df['longitude'], 99.95))]

In [105]:
# Change the created column to datetime
df['created'] = pd.to_datetime(df['created'])
df.dtypes

bathrooms                      float64
bedrooms                         int64
created                 datetime64[ns]
description                     object
display_address                 object
latitude                       float64
longitude                      float64
price                            int64
street_address                  object
interest_level                  object
elevator                         int64
cats_allowed                     int64
hardwood_floors                  int64
dogs_allowed                     int64
doorman                          int64
dishwasher                       int64
no_fee                           int64
laundry_in_building              int64
fitness_center                   int64
pre-war                          int64
laundry_in_unit                  int64
roof_deck                        int64
outdoor_space                    int64
dining_room                      int64
high_speed_internet              int64
balcony                  

In [106]:
# Engineer at least 2 new features
perks = df.columns[10:]

df['total_perks'] = df[perks].sum(axis=1)
df['total_rooms'] = df['bathrooms'] + df['bedrooms']

In [107]:
# Do train/test split

# Train (April & May)
train_months = (df['created'].dt.month == 4) | (df['created'].dt.month == 5)
train = df[train_months]

# Test (June)
test_month = df['created'].dt.month == 6
test = df[test_month]

### Fit a Linear Regression model with at least two features
>I'm using 4, but first I want to start with a baseline

In [116]:
# Choose a model class
from sklearn.linear_model import LinearRegression

# Instantiate the class
model = LinearRegression()

# Arrange the data into features matrix and target vector
features = ['total_rooms', 'total_perks', 'latitude', 'longitude']
target = 'price'
X_train = train[features]
y_train = train[target]

# Fit the model
model.fit(X_train, y_train);

In [117]:
# Get the model's coefficient and intercept
print("Intercept:", model.intercept_)
coefficients = pd.Series(model.coef_, features)
print(coefficients.to_string())

Intercept: -1243678.4115042493
total_rooms      792.945481
total_perks       68.310265
latitude        2260.008839
longitude     -15582.319644


In [120]:
# Get regression metrics for the train and test data (MAE, RMSE, r2)
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

X_test = test[features]
y_test = test[target]

# Define a function that prints out the metrics
def metrics(X, y):
    y_pred = model.predict(X)
    mae = mean_absolute_error(y, y_pred)
    rmse  = np.sqrt(mean_squared_error(y, y_pred))
    r2 = r2_score(y, y_pred)
    output_string = f"\n\tMAE: {mae: .2f}\n"\
                    f"\tRMSE: {rmse: .2f}\n"\
                    f"\tR^2: {r2: .2f}\n"
    return output_string

print("Train Metrics:", metrics(X_train, y_train))
print("Test Metrics:", metrics(X_test, y_test))

Train Metrics: 
	MAE:  789.60
	RMSE:  1227.74
	R^2:  0.51

Test Metrics: 
	MAE:  799.50
	RMSE:  1223.35
	R^2:  0.52

